# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
E. Matthews  ->  E. Matthews  |  ['E. Matthews']
T. Henning  ->  T. Henning  |  ['T. Henning']
J. Bouwman  ->  J. Bouwman  |  ['J. Bouwman']
S. Scheithauer  ->  S. Scheithauer  |  ['S. Scheithauer']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
K. Doi  ->  K. Doi  |  ['K. Doi']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
H. Jiang  ->  H. Jiang  |  ['H. Jiang']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
R. Seeburger  ->  L. R. Seeburger  |  ['R. Seeburger']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
R. Burn  ->  R. Burn  |  ['R. Burn']


Y. Wang  ->  Y. Wang  |  ['Y. Wang']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
N. Mackensen  ->  N. Mackensen  |  ['N. Mackensen']
Arxiv has 114 new papers today
          11 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/11 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.08958


extracting tarball to tmp_2507.08958...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.08961
extracting tarball to tmp_2507.08961...

 done.


...

/tmp/ipykernel_3395/2822249172.py:52: LatexWarning: 2507.09874 did not run properly
Unable to locate Ghostscript on paths
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2507.09971


extracting tarball to tmp_2507.09971...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.10314


extracting tarball to tmp_2507.10314... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.08961-b31b1b.svg)](https://arxiv.org/abs/2507.08961) | **JWST/MIRI observations of the young TWA 27 system: hydrocarbon disk chemistry, silicate clouds, evidence for a CPD**  |
|| P. Patapis, et al. -- incl., <mark>E. Matthews</mark>, <mark>T. Henning</mark>, <mark>J. Bouwman</mark>, <mark>S. Scheithauer</mark> |
|*Appeared on*| *2025-07-15*|
|*Comments*| *Submitted to Astronomy&Astrophysics*|
|**Abstract**|            The Mid-Infrared Instrument (MIRI) on the James Webb Space Telescope (JWST) enables the characterisation of young self-luminous gas giants at previously inaccessible wavelengths, revealing physical processes in gas, dust, and clouds. We characterise the young planetary system TWA 27 (2M1207) in the mid-infrared (MIR), studying the atmosphere and disk spectra of the M9 brown dwarf TWA 27A and its L6 planetary-mass companion TWA 27b. We obtained data with the MIRI Medium Resolution Spectrometer (MRS) from 4.9 to 20 um, and MIRI Imaging in the F1000W and F1500W filters. We applied high-contrast imaging methods to extract the companion's spectral energy distribution up to 15 um at 0.78 arcsec separation and a contrast of 60. Combining these with published JWST/NIRSpec spectra, we analysed the 1-20 um range using self-consistent atmospheric grids and 0D slab models for molecular disk emission. The atmosphere of TWA 27A is well fitted by a BT-SETTL model with Teff 2780 K, log g 4.3, plus a 740 K blackbody for the inner disk rim. The disk shows at least 11 organic molecules, with no water or silicate dust emission detected. The atmosphere of TWA 27b is matched by a Teff 1400 K low-gravity model with extinction, best fit by the ExoREM grid. MIRI spectra and photometry for TWA 27b reveal a silicate cloud absorption feature between 8-10 um and significant (>5 sigma) infrared excess at 15 um consistent with circumplanetary disk emission. These MIRI observations provide new insights into TWA 27, revealing diverse features to study the formation and evolution of circumplanetary disks and young dusty atmospheres.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.09343-b31b1b.svg)](https://arxiv.org/abs/2507.09343) | **Detailed Architecture of the L 98-59 System and Confirmation of a Fifth Planet in the Habitable Zone**  |
|| C. Cadieux, et al. -- incl., <mark>E.-M. Ahrer</mark> |
|*Appeared on*| *2025-07-15*|
|*Comments*| *32 pages, 12 figures, 8 tables, accepted for publication in AJ*|
|**Abstract**|            The L 98-59 system, identified by TESS in 2019, features three transiting exoplanets in compact orbits of 2.253, 3.691, and 7.451 days around an M3V star, with an outer 12.83-day non-transiting planet confirmed in 2021 using ESPRESSO. The planets exhibit a diverse range of sizes (0.8-1.6 R$_{\oplus}$), masses (0.5-3 M$_{\oplus}$), and likely compositions (Earth-like to possibly water-rich), prompting atmospheric characterization studies with HST and JWST. Here, we analyze 16 new TESS sectors and improve radial velocity (RV) precision of archival ESPRESSO and HARPS data using a line-by-line framework, enabling stellar activity detrending via a novel differential temperature indicator. We refine the radii of L 98-59 b, c, and d to 0.837 $\pm$ 0.019 R$_{\oplus}$, 1.329 $\pm$ 0.029 R$_{\oplus}$, 1.627 $\pm$ 0.041 R$_{\oplus}$, respectively. Combining RVs with transit timing variations (TTV) of L 98-59 c and d from TESS and JWST provides unprecedented constraints on the masses and eccentricities of the planets. We report updated masses of 0.46 $\pm$ 0.11 M$_{\oplus}$ for b, 2.00 $\pm$ 0.13 M$_{\oplus}$ for c, and 1.64 $\pm$ 0.07 M$_{\oplus}$ for d, and a minimum mass of 2.82 $\pm$ 0.19 M$_{\oplus}$ for e. We additionally confirm L 98-59\,f, a non-transiting super-Earth with a minimal mass of 2.80 $\pm$ 0.30 M$_{\oplus}$ on a 23.06-day orbit inside the Habitable Zone. The TTVs of L 98-59 c and d (<3 min, $P_{\rm TTV} = 396$ days) constrain the eccentricities of all planets to near-circular orbits ($e \lesssim 0.04$). An internal structure analysis of the transiting planets reveals increasing water-mass fractions ($f_{\rm H_{2}O}$) with orbital distance, reaching $f_{\rm H_{2}O} \approx 0.16$ for L 98-59\ d. We predict eccentricity-induced tidal heating in L 98-59 b with heat fluxes comparable to those of Io, potentially driving volcanic activity.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.09402-b31b1b.svg)](https://arxiv.org/abs/2507.09402) | **Leaky dust trap in the PDS 70 disk revealed by ALMA Band 9 observations**  |
|| A. Sierra, et al. -- incl., <mark>M. Benisty</mark>, <mark>K. Doi</mark>, <mark>D. Fasano</mark> |
|*Appeared on*| *2025-07-15*|
|*Comments*| *12 pages*|
|**Abstract**|            We present new observations of the PDS 70 disc obtained with the Atacama Large Millimeter/sub-millimeter Array (ALMA) in Band 9 (671 GHz) at 0.242$^{\prime\prime}$ resolution, which provide valuable insights into the spatial distribution of sub-millimetre grains in the disc. The data reveal a ring-like morphology, with a radial peak located between those previously observed at infrared wavelengths and longer millimetre observations. Additionally, we detect a tentative outer shoulder in Band 9 that is not observed at longer wavelengths. These findings suggest that small grains ($\sim 100 \mu$m) traced by Band 9 may be escaping from the pressure bump both radially inwards and outwards, or may be tracing different disc layers than those probed at longer wavelengths. A multi-wavelength analysis of the disc at millimetre wavelengths and the best fit to the spectral energy distribution shows the presence of centimetre grains around the ring location, where the dust surface density also peaks, compatible with dust trap models. The grain size in the disc cavity is not well constrained but is consistent with grains as small as 10 $\mu$m, supporting the hypothesis that small dust grain filters through the cavity. We use dust evolution models to demonstrate that a turbulent viscosity of $\alpha \gtrsim 10^{-3}$ allows small grains to filter through the disc gap, while $\alpha \lesssim 5 \times 10^{-3}$ is required to retain large grains in the pressure bump. The Band 9 observations of PDS 70 validate theoretical models and confirm the presence of pebble flux through the disc gap.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.09622-b31b1b.svg)](https://arxiv.org/abs/2507.09622) | **Millions of Main-Sequence Binary Stars from Gaia BP/RP Spectra**  |
|| <mark>J. Li</mark>, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>J. Müller-Horn</mark>, <mark>K. El-Badry</mark>, <mark>R. Seeburger</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-07-15*|
|*Comments*| *14 pages, 15 figures, submitted to A&A*|
|**Abstract**|            We present the main-sequence binary (MSMS) Catalog derived from Gaia Data Release 3 BP/RP (XP) spectra. Leveraging the vast sample of low-resolution Gaia XP spectra, we develop a forward modeling approach that maps stellar mass and photometric metallicity to XP spectra using a neural network. Our methodology identifies binary systems through statistical comparison of single- and binary-star model fits, enabling detection of binaries with mass ratios between 0.4 and 1.0 and flux ratios larger than 0.1. From an initial sample of 35 million stars within 1 kpc, we identify 14 million binary candidates and define a high-confidence "golden sample" of 1 million binary systems. This large, homogeneous sample enables detailed statistical analysis of binary properties across diverse Galactic environments, providing new insights into binary star formation and evolution. In addition, the $\chi^2$ comparison allows us to distinguish stars with luminous companions from single stars or binaries with dark companions, such as white dwarfs, neutron stars and black hole candidates, improving our understanding of compact object populations.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.09430-b31b1b.svg)](https://arxiv.org/abs/2507.09430) | **MVPinn: Integrating Milne-Eddington Inversion with Physics-Informed Neural Networks for GST/NIRIS Observations**  |
|| Q. Li, et al. -- incl., <mark>H. Jiang</mark> |
|*Appeared on*| *2025-07-15*|
|*Comments*| **|
|**Abstract**|            We introduce MVPinn, a Physics-Informed Neural Network (PINN) approach tailored for solving the Milne-Eddington (ME) inversion problem, specifically applied to spectropolarimetric observations from the Big Bear Solar Observatory's Near-InfraRed Imaging Spectropolarimeter (BBSO/NIRIS) at the Fe I 1.56 {\mu}m lines. Traditional ME inversion methods, though widely used, are computationally intensive, sensitive to noise, and often struggle to accurately capture complex profile asymmetries resulting from gradients in magnetic field strength, orientation, and line-of-sight velocities. By embedding the ME radiative transfer equations directly into the neural network training as physics-informed constraints, our MVPinn method robustly and efficiently retrieves magnetic field parameters, significantly outperforming traditional inversion methods in accuracy, noise resilience, and the ability to handle asymmetric and weak polarization signals. After training, MVPinn infers one magnetogram in about 15 seconds, compared to tens of minutes required by traditional ME inversion on high-resolution spectropolarimetric data. Quantitative comparisons demonstrate excellent agreement with well-established magnetic field measurements from the SDO/HMI and Hinode/SOT-SP instruments, with correlation coefficients of approximately 90%. In particular, MVPINN aligns better with Hinode/SOT-SP data, indicating some saturation of HMI data at high magnetic strengths. We further analyze the physical significance of profile asymmetries and the limitations inherent in the ME model assumption. Our results illustrate the potential of physics-informed machine learning methods in high-spatial-temporal solar observations, preparing for more sophisticated, real-time magnetic field analysis essential for current and next-generation solar telescopes and space weather monitoring.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.09552-b31b1b.svg)](https://arxiv.org/abs/2507.09552) | **Probing the scalar-induced gravitational waves with the Five-hundred-meter Aperture Spherical radio Telescope and the Square Kilometer Array**  |
|| <mark>J. Li</mark>, G.-H. Guo, Y. Zu |
|*Appeared on*| *2025-07-15*|
|*Comments*| **|
|**Abstract**|            Gravitational wave astronomy presents a promising opportunity to directly observe scalar-induced gravitational waves originating from the early universe. Future experiments, including ground-based interferometers like LIGO and Virgo, the Pulsar Timing Array, and telescopes such as FAST and SKA, are poised to significantly enhance sensitivity to these gravitational waves. In this paper, we combined Cosmic Microwave Background data with upper or lower limits of the stochastic gravitational wave background provided by FAST or SKA, to constrain scalar-induced gravitational waves. To provide a comprehensive forecast, we consider two scenarios: one where FAST or SKA does not detect scalar-induced gravitational waves, thereby setting an upper limit on the fractional energy density; and another where these waves are detected successfully, thus establishing a lower limit on the fractional energy density. In the $\Lambda$CDM+$r$ model, the scalar spectral index of the power-law power spectrum is constrained to $n_s=0.9589^{+0.0021}_{-0.0011}$ from the combinations of CMB+BAO+SKA datasets in the upper limit scenario. The constraint shifts to $n_s = 0.9661^{+0.0027}_{-0.0039}$ in the lower limit scenario. Comparing with the constraint from the combinations of CMB+BAO datasets, the scalar spectral index $n_s$ exhibits significant changes, which could serve as an indicator for detecting scalar-induced gravitational waves. In the $\Lambda$CDM+$\alpha_s$+$r$ model and the $\Lambda$CDM+$\alpha_s$+$\beta_s$+$r$ model, the running of the scalar spectral index $\alpha_s$ and the running of the running of the scalar spectral index $\beta_s$ also show notable variations, suggesting potential indicators. The numerical findings clearly demonstrate the impact of the upper and lower limits provided by FAST or SKA.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.09971-b31b1b.svg)](https://arxiv.org/abs/2507.09971) | **Noema formIng Cluster survEy (NICE): A Census of Star Formation and Cold Gas Properties in Massive protoclusters at 1.5<z<4**  |
|| L. Zhou, et al. -- incl., <mark>Y. Wang</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-07-15*|
|*Comments*| *12 pages, 7 figures, 1 table and 1 figure in appendix. accepted by A&A*|
|**Abstract**|            Massive protoclusters at z~1.5-4, the peak of the cosmic star formation history, are key to understanding the formation mechanisms of massive galaxies in today's clusters. However, studies of protoclusters at these high redshifts remain limited, primarily due to small sample sizes and heterogeneous selection criteria. In this work, we conduct a systematic investigation of the star formation and cold gas properties of member galaxies of eight massive protoclusters in the COSMOS field, using the statistical and homogeneously selected sample from the Noema formIng Cluster survEy (NICE). Our analysis reveals a steep increase in the star formation rates per halo mass ($\Sigma_{\rm SFR} /M_{\rm halo}$) with redshifts in these intensively star-forming protoclusters, reaching values one to two orders of magnitude higher than those observed in the field at z>2. We further show that, instead of an enhancement of starbursts, this increase is largely driven by the concentration of massive and gas-rich star-forming galaxies in the protocluster cores. The member galaxies still generally follow the same star formation main sequence as in the field, with a moderate enhancement at the low mass end. Notably, the most massive protocluster galaxies ($M_{\rm halo}$>8$\times$10$^{10}$M$_\odot$) exhibit higher $f_{\rm gas}$ and $\tau_{\rm gas}$ than their field counterparts, while remaining on the star forming main sequence. These gas-rich, massive, and star-forming galaxies are predominantly concentrated in the protocluster cores and are likely progenitors of massive ellipticals in the center of today's clusters. These results suggest that the formation of massive galaxies in such environments is sustained by substantial gas reservoirs, which support persistent star formation and drive early mass assembly in forming cluster cores.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.10314-b31b1b.svg)](https://arxiv.org/abs/2507.10314) | **How an overweight and rapidly rotating PG 1159 star in the Galactic halo challenges evolutionary models**  |
|| <mark>N. Mackensen</mark>, et al. |
|*Appeared on*| *2025-07-15*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            PG 1159 stars are thought to be progenitors of the majority of H-deficient white dwarfs. Their unusual He-, C-, and O-dominated surface composition is typically believed to result from a late thermal pulse experienced by a single (pre-)white dwarf. Yet, other formation channels - involving close binary evolution - have recently been proposed and could lead to similar surface compositions. Here we present a non-local thermodynamic equilibrium spectral analysis based on new UV and archival optical spectra of one of the hottest PG 1159 stars, $\text{RX J}0122.9\text{ -}7521$. We find $T_\text{eff} = 175$ kK and a surface gravity of log $g = 7.7$, and an astonishingly low O/C ratio of $7.3 \times 10^{-3}$ by mass. By combining the spectroscopic surface gravity and Gaia parallax with a spectral energy distribution fit, we derive a mass of $M_\text{spec} = 1.8^{+1.1}_{-0.7}$ $M_\odot$. Although this spectroscopic mass is higher than predicted by evolutionary models, it is subject to substantial uncertainty. Furthermore, we find that $\text{RX J}0122.9\text{ -}7521$ shows strongly rotationally broadened lines, suggesting that the previously reported photometric period of $41$ min indeed corresponds to the rotational period of this star. Our kinematic analysis shows that $\text{RX J}0122.9\text{ -}7521$ belongs to the Galactic halo, which - assuming single-star evolution - is in stark contrast to its relatively high mass. The rapid rotation, high mass, and halo kinematics, as well as the lack of evidence for a close companion, lead us to believe that $\text{RX J}0122.9\text{ -}7521$ formed through the merger of two white dwarfs. Yet, none of the current models can explain the surface abundances of $\text{RX J}0122.9\text{ -}7521$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.09874-b31b1b.svg)](https://arxiv.org/abs/2507.09874) | **The New Generation Planetary Population Synthesis (NGPPS) VIII. Impact of host star metallicity on planet occurrence rates, orbital periods, eccentricities, and radius valley morphology**  |
|| D.-C. Chen, et al. -- incl., <mark>R. Burn</mark> |
|*Appeared on*| *2025-07-15*|
|*Comments*| *20 Pages, 23 figures, 1 Table, Accepted for publication on A&A*|
|**Abstract**|            The dust-to-gas ratio in the protoplanetary disk, which is likely imprinted into the host star metallicity, is a property that plays a crucial role during planet formation. We aim at constraining planet formation and evolution processes by statistically analysing planetary systems generated by the Generation III Bern model, comparing with the correlations derived from observational samples. Using synthetic planets biased to observational completeness, we find that (1) the occurrence rates of large giant planets and Neptune-size planets are positively correlated with [Fe/H], while small sub-Earths exhibit an anti-correlation. In between, for sub-Neptune and super-Earth, the occurrence rate first increases and then decreases with increasing [Fe/H] with an inflection point at 0.1 dex. (2) Planets with orbital periods shorter than ten days are more likely to be found around stars with higher metallicity, and this tendency weakens with increasing planet radius. (3) Both giant planets and small planets exhibit a positive correlation between the eccentricity and [Fe/H], which could be explained by the self-excitation and perturbation of outer giant planets. (4) The radius valley deepens and becomes more prominent with increasing [Fe/H], accompanied by a lower super-Earth-to-sub-Neptune ratio. Furthermore, the average radius of the planets above the valley increases with [Fe/H]. Our nominal model successfully reproduces many observed correlations with stellar metallicity, supporting the description of physical processes and parameters included in the Bern model. However, the dependences of orbital eccentricity and period on [Fe/H] predicted by the synthetic population is however significantly weaker than observed. This discrepancy suggests that long-term dynamical interactions between planets, along with the impact of binaries/companions, can drive the system towards a dynamically hotter state.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to locate Ghostscript on paths</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.08961/./01_twa-27_A+b_spectrum.png', 'tmp_2507.08961/./03_twa27_A_modelfit.png', 'tmp_2507.08961/./04_twa27_b_modelfit.png']
copying  tmp_2507.08961/./01_twa-27_A+b_spectrum.png to _build/html/
copying  tmp_2507.08961/./03_twa27_A_modelfit.png to _build/html/
copying  tmp_2507.08961/./04_twa27_b_modelfit.png to _build/html/
exported in  _build/html/2507.08961.md
    + _build/html/tmp_2507.08961/./01_twa-27_A+b_spectrum.png
    + _build/html/tmp_2507.08961/./03_twa27_A_modelfit.png
    + _build/html/tmp_2507.08961/./04_twa27_b_modelfit.png
found figures ['tmp_2507.09343/./MR_crop.png', 'tmp_2507.09343/./corner_L9859b_rocky_crop.png', 'tmp_2507.09343/./corner_L9859c_crop.png', 'tmp_2507.09343/./corner_L9859d_crop.png', 'tmp_2507.09343/./orbits_crop.png', 'tmp_2507.09343/./HZ_crop.png']
copying  tmp_2507.09343/./MR_crop.png to _build/html/
copying  tmp_2507.09343/./corner_L9859b_rocky_crop.png to _build/html/
copying  tmp_2507.09343/./corner_L9859c_crop.png to _bu

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\todo}[1]{\textcolor{red}{#1}}$
$\newcommand{\danny}{\textcolor{green}}$
$\newcommand{\mmc}{\textcolor{purple}}$
$\newcommand{\mum}{\mum\xspace}$
$\newcommand{\ama}[1]{\textcolor{red}{\rm #1^{(\rm AMA)}}}$
$\newcommand{\Lsun}{L_\odot}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\MJ}{M_\mathrm{J}}$
$\newcommand{\Mp}{M_\mathrm{p}}$
$\newcommand{\Macc}{\dot{M}_\mathrm{acc}}$
$\newcommand{\RJ}{R_\mathrm{J}}$
$\newcommand{\Rp}{R_\mathrm{p}}$
$\newcommand{\Teff}{T_{\text{eff}}}$
$\newcommand{\MJyr}{\MJ \mathrm{yr}^{-1}}$
$\newcommand{\Rd}{R_\mathrm{dust}}$
$\newcommand{\Rcav}{R_\mathrm{cav}}$
$\newcommand{\Rout}{R_\mathrm{out}}$
$\newcommand{\Rg}{R_\mathrm{gas}}$
$\newcommand{\Lacc}{L_\mathrm{acc}}$
$\newcommand{\LHa}{L_\mathrm{H\alpha}}$
$\newcommand{\FHa}{F_\mathrm{H\alpha}}$
$\newcommand{\AHa}{A_\mathrm{H\alpha}}$
$\newcommand{\Av}{A_\mathrm{V}}$
$\newcommand{\MaccAv}{\overline{\dot{M}_\mathrm{acc}}}$
$\newcommand{\c2h2}{C_\mathrm{2}H_\mathrm{2}\xspace}$
$\newcommand{\c6h6}{C_\mathrm{6}H_\mathrm{6}\xspace}$
$\newcommand{\c2h6}{C_\mathrm{2}H_\mathrm{6}\xspace}$
$\newcommand{\co2}{CO_\mathrm{2}\xspace}$</div>



<div id="title">

# JWST/MIRI observations of the young TWA 27 system: hydrocarbon disk chemistry, silicate clouds, evidence for a CPD

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.08961-b31b1b.svg)](https://arxiv.org/abs/2507.08961)<mark>Appeared on: 2025-07-15</mark> -  _Submitted to Astronomy&Astrophysics_

</div>
<div id="authors">

P. Patapis, et al. -- incl., <mark>E. Matthews</mark>, <mark>T. Henning</mark>, <mark>J. Bouwman</mark>, <mark>S. Scheithauer</mark>

</div>
<div id="abstract">

**Abstract:** The Mid-Infrared Instrument (MIRI) on board the James Webb Space Telescope (JWST) enables the characterisation of young self-luminous gas giants in a wavelength range previously inaccessible, revealing insights into physical processes of the gas, dust, and clouds. We aim to characterise the young planetary system TWA 27 (2M1207) in the mid-infrared (MIR), revealing the atmosphere and disk spectra of the M9 brown dwarf TWA 27A and its L6 planetary mass companion TWA 27b. We obtain data from the MIRI Medium Resolution Spectrometer (MRS) from 4.9 to 20 $\mum$ , and MIRI Imaging in the F1000W and F1500W filters. We apply high-contrast-imaging data processing methods in order to extract the companion spectral energy distribution up to 15 $\mum$ at a separation of 0 $\farcs$ 78 and contrast of 60. Using published spectra from JWST/NIRSpec, we analyse the 1-20 $\mum$ spectra with self-consistent atmosphere grids, and the molecular disk emission from TWA 27A with 0D slab models. We find that the atmosphere of TWA 27A is well fitted with the \texttt{BT-SETTL} model of effective temperature $T_{\text{eff}}\sim 2780$ K, $\mathrm{log} g \sim 4.3$ , and a blackbody component of $\sim$ 740 K for the circumstellar disk inner rim. The disk consists of at least 11 organic molecules, and neither water or silicate dust emission are detected.   The atmosphere of the planet TWA 27b matches with a $T_{\text{eff}}\sim$ 1400 K low gravity model when adding extinction, with the \texttt{ExoREM} grid fitting the best. MIRI spectra and photometry for TWA 27b reveal a silicate cloud absorption feature between 8-10 $\mum$ , and evidence (>5 $\sigma$ ) for infrared excess at 15 $\mum$ that is consistent with predictions from circumplanetary disk emission. The MIRI observations present novel insights into the young planetary system TWA 27, showing a diversity of features that can be studied to understand the formation and evolution of circumplanetary disks and young dusty atmospheres.

</div>

<div id="div_fig1">

<img src="tmp_2507.08961/./01_twa-27_A+b_spectrum.png" alt="Fig2" width="100%"/>

**Figure 2. -** Complete 1-20 $\mum$ spectra of TWA 27A (a) and TWA 27b (d) with JWST/NIRSpec and JWST/MIRI. Details of TWA 27A are presented in panel (b) for the instrument overlap between NIRSpec and MIRI, and in panel (c) for the molecular emission of the TWA 27A disk. Error bars for the NIRSpec spectra, and MIRI spectrum of TWA 27A are not visible. (*fig:twa-27-A*)

</div>
<div id="div_fig2">

<img src="tmp_2507.08961/./03_twa27_A_modelfit.png" alt="Fig4" width="100%"/>

**Figure 4. -** Model fit to TWA 27A. Top: data and model fit using the \texttt{BT-SETTL} grid. The atmosphere model is shown in gray, the blackbody disk component in red, and the combined model in gold. Middle: residuals between data and model in physical units (mJy). Bottom: details of model fit for two regions of the spectrum. (*fig:twa-27-A_modelfit*)

</div>
<div id="div_fig3">

<img src="tmp_2507.08961/./04_twa27_b_modelfit.png" alt="Fig5" width="100%"/>

**Figure 5. -** Model fit to TWA 27b. Top: data and model fit using the \texttt{ATMO}, \texttt{ExoREM}, and \texttt{SONORA} grids, including extinction and an offset of 0.15 mJy for visibility. In order to explain the 15 $\mum$ IR excess due to a cold disk a blackbody component (350 K) is shown in magenta and added to the \texttt{ExoREM} flux. Middle: residuals between data and model in physical units (mJy). The discrepancy of data and models at 7.5-10 $\mum$ is most likely attributed to a silicate cloud. Bottom: details of model fit for two regions of the spectrum, including the methane and CO bands, only showing \texttt{ExoREM}. (*fig:twa-27-b_modelfit*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.08961"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\todo}[1]{\textcolor{orange}{#1}}$
$\newcommand{\tofix}[1]{\textcolor{red}{#1}}$
$\newcommand{\dtemp}{d\textit{Temp}}$
$\newcommand$</div>



<div id="title">

# Detailed Architecture of the L 98-59 System and Confirmation of a Fifth Planet in the Habitable Zone

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.09343-b31b1b.svg)](https://arxiv.org/abs/2507.09343)<mark>Appeared on: 2025-07-15</mark> -  _32 pages, 12 figures, 8 tables, accepted for publication in AJ_

</div>
<div id="authors">

C. Cadieux, et al. -- incl., <mark>E.-M. Ahrer</mark>

</div>
<div id="abstract">

**Abstract:** The L 98-59 system, identified by TESS in 2019, features three transiting exoplanets in compact orbits of 2.253, 3.691, and 7.451 days around an M3V star, with an outer 12.83-day non-transiting planet confirmed in 2021 using ESPRESSO. The planets exhibit a diverse range of sizes (0.8--1.6 R $_{\oplus}$ ), masses (0.5--3 M $_{\oplus}$ ), and likely compositions (Earth-like to possibly water-rich), prompting atmospheric characterization studies with HST and JWST. Here, we analyze 16 new TESS sectors and improve radial velocity (RV) precision of archival ESPRESSO and HARPS data using a line-by-line framework, enabling stellar activity detrending via a novel differential temperature indicator. We refine the radii of L 98-59 b, c, and d to 0.837 $\pm$ 0.019 R $_{\oplus}$ , 1.329 $\pm$ 0.029 R $_{\oplus}$ , 1.627 $\pm$ 0.041 R $_{\oplus}$ , respectively. Combining RVs with transit timing variations (TTV) of L 98-59 c and d from TESS and JWST provides unprecedented constraints on the masses and eccentricities of the planets. We report updated masses of 0.46 $\pm$ 0.11 M $_{\oplus}$ for b, 2.00 $\pm$ 0.13 M $_{\oplus}$ for c, and 1.64 $\pm$ 0.07 M $_{\oplus}$ for d, and a minimum mass of 2.82 $\pm$ 0.19 M $_{\oplus}$ for e. We additionally confirm L 98-59 f, a non-transiting super-Earth with a minimal mass of 2.80 $\pm$ 0.30 M $_{\oplus}$ on a 23.06-day orbit inside the Habitable Zone. The TTVs of L 98-59 c and d ( $<$ 3 min, $P_{\rm TTV} = 396$ days) constrain the eccentricities of all planets to near-circular orbits ( $e \lesssim 0.04$ ). An internal structure analysis of the transiting planets reveals increasing water-mass fractions ( $f_{\rm H_{2}O}$ ) with orbital distance, reaching $f_{\rm H_{2}O} \approx 0.16$ for L 98-59 d. We predict eccentricity-induced tidal heating in L 98-59 b with heat fluxes comparable to those of Io, potentially driving volcanic activity.

</div>

<div id="div_fig1">

<img src="tmp_2507.09343/./MR_crop.png" alt="Fig4" width="100%"/>

**Figure 4. -** Mass--radius constraints on the L 98-59 planets (b: green, c: purple, d: red, e: blue, f: teal) with other exoplanets around M dwarfs in the background (gray points). The previous values from \citetalias{Demangeon_2021} are shown in black attached to revised measurements. For L 98-59 e and f, 1- and 2-$\sigma$ confidence contours are shown with the radius estimated using \texttt{spright}\citep{Parviainen_2024}  and with a cross symbol showing the two modes (rocky and volatile-rich). Rocky composition curves from \cite{Zeng_2019} are drawn from pure iron to pure silicate rocks. Water-rich interior models at $T = 400$ K from \cite{Aguichine_2021} are shown with dotted blue curves. The orange region delimits a degeneracy ($H_2$- or $H_2$O-rich) in composition \citep{Rogers_2023}. (*fig:mr*)

</div>
<div id="div_fig2">

<img src="tmp_2507.09343/./corner_L9859b_rocky_crop.png" alt="Fig11.1" width="33%"/><img src="tmp_2507.09343/./corner_L9859c_crop.png" alt="Fig11.2" width="33%"/><img src="tmp_2507.09343/./corner_L9859d_crop.png" alt="Fig11.3" width="33%"/>

**Figure 11. -** Marginalised 1D and 2D posterior distributions of internal structure parameters for L 98-59 b (left), c (center), and d (right) derived with \texttt{smint}\citep{Piaulet_2023}. The 2D posteriors display the 1- and 2-$\sigma$ contours. The analysis constrains the core- and water-mass fractions ($f^{\prime}_{\rm core}$, $f_{\rm H_{2}O}$) using priors on planetary mass ($M_{\rm p}$) and irradiation temperature ($T_{\rm irr}$; not shown), while requiring consistency with the observed planetary radius ($R_{\rm p}$). The sub-Earth L 98-59 b is assumed to be purely rocky ($f_{\rm H_{2}O} = 0$). We explored a pure rocky model for L 98-59 c and water-rich models for L 98-59 c and d with either a uniform prior on $f^{\prime}_{\rm core}$(`up') or using $f^{\prime}_{\rm core}$ from planet b as a prior (`bp'). L 98-59 b has a $f^{\prime}_{\rm core}$ consistent with a wide range of iron content in the interior (from coreless to super-Mercury), with a mode close to Earth's value ($f^{\prime}_{\rm core} = 0.325$, \citealt{Wang_2018}). L 98-59 c likely has a small $f_{\rm H_{2}O} \sim 0.03$ to explain its mass and radius. For L 98-59 d, we infer a statistically significant $f_{\rm H_{2}O}$ = 0.16--0.18 irrespective of the choice of prior. (*fig:interior*)

</div>
<div id="div_fig3">

<img src="tmp_2507.09343/./orbits_crop.png" alt="Fig1.1" width="50%"/><img src="tmp_2507.09343/./HZ_crop.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** _Top_: Above view of the L 98-59 planetary system on BJD = 2 460 000 assuming circular orbits. The line of sight towards Earth is to the right. The Habitable Zone defined in \citealt{Kopparapu_2013} is shown in green for runaway/maximum greenhouse (conservative) and pale green for early recent Venus/early Mars (optimistic). _Bottom_: Comparative exoplanetology of M-dwarf multiplanetary systems ordered by stellar effective temperature and planetary instellation. An arbitrary shift of $+$100 K in $T_{\rm eff}$ has been applied to \hbox{TOI-700} and Teegarden's star for clarity. In both panels, transiting (non-transiting) exoplanets are shown as filled (open) circles, with circle size proportional to planetary radius. For planets with only a minimum mass constraint, the radius is set to the 95$^{\rm th}$ percentile of the prediction from \texttt{spright}\citep{Parviainen_2024}. (*fig:orbit*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.09343"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\anibal}[1]{{\color{orange} [AS: #1]}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Leaky dust trap in the PDS 70 disk revealed by ALMA Band 9 observations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.09402-b31b1b.svg)](https://arxiv.org/abs/2507.09402)<mark>Appeared on: 2025-07-15</mark> -  _12 pages_

</div>
<div id="authors">

A. Sierra, et al. -- incl., <mark>M. Benisty</mark>, <mark>K. Doi</mark>, <mark>D. Fasano</mark>

</div>
<div id="abstract">

**Abstract:** We present new observations of the PDS 70 disc obtained with the Atacama Large Millimeter/sub-millimeter Array (ALMA) in Band 9 (671 GHz) at 0.242 $^{\prime\prime}$ resolution, which provide valuable insights into the spatial distribution of sub-millimetre grains in the disc. The data reveal a ring-like morphology, with a radial peak located between those previously observed at infrared wavelengths and longer millimetre observations. Additionally, we detect a tentative outer shoulder in Band 9 that is not observed at longer wavelengths. These findings suggest that small grains ( $\sim 100 \mu$ m) traced by Band 9 may be escaping from the pressure bump both radially inwards and outwards, or may be tracing different disc layers than those probed at longer wavelengths.A multi-wavelength analysis of the disc at millimetre wavelengths and the best fit to the spectral energy distribution shows the presence of centimetre grains around the ring location, where the dust surface density also peaks, compatible with dust trap models. The grain size in the disc cavity is not well constrained but is consistent with grains as small as 10 $\mu$ m, supporting the hypothesis that small dust grain filters through the cavity.We use dust evolution models to demonstrate that a turbulent viscosity of $\alpha \gtrsim 10^{-3}$ allows small grains to filter through the disc gap, while $\alpha \lesssim 5 \times 10^{-3}$ is required to retain large grains in the pressure bump. The Band 9 observations of PDS 70 validate theoretical models and confirm the presence of pebble flux through the disc gap.

</div>

<div id="div_fig1">

<img src="tmp_2507.09402/./Figures/Results.png" alt="Fig8" width="100%"/>

**Figure 8. -** Visibility modelling of the disc around PDS 70 at different wavelengths.
    Left: tclean image of the dust continuum data. Middle left: tclean image of the symmetric dust continuum data. Middle right: tclean image of the Frankenstein model. Right: tclean map of the residual visibilities (Symmetric - Model). The white iso-contour show the region with residuals above/below $\pm 5\sigma$.
    The units of all colour bars are mJy beam$^{-1}$.
    Right panels show the real part of the continuum visibilities (blue dots) as a function of the uv-distance (baseline). The flux units in all right panels are mJy.
    The solid black line shows the visibility Frankenstein model.
     (*fig:visibility_modelling*)

</div>
<div id="div_fig2">

<img src="tmp_2507.09402/./Figures/Radial_profiles.png" alt="Fig9" width="100%"/>

**Figure 9. -** Left: Radial profiles of the brightness temperature obtained from the visibility fit at different ALMA wavelengths (see colour code). The vertical dashed-dotted lines indicate the orbits of PDS 70b, and PDS 70c.
    Right: Zoom-in around the ring peaks. The grey dashed line shows the normalised brightness radial profile from infrared observations at H band ($\lambda = 1.625 \mu$m). The radial positions of the peaks are shown as vertical dashed lines. (*fig:BrightnessTemperature*)

</div>
<div id="div_fig3">

<img src="tmp_2507.09402/./Figures/Band9_Map.png" alt="Fig7" width="100%"/>

**Figure 7. -** Left: Dust continuum image at ALMA Band 9 (671 GHz).
    Middle: Azimuthal profile around the ring. The arrows indicate the azimuth of the northwest and southeast major axes.
    Right: Brightness temperature radial profile of PDS 70 extracted from the ALMA images at different wavelengths: 100 GHz (red), 220 GHz (yellow), 260 GHz (green), 350 GHz (blue), 671 GHz (purple). The horizontal lines show the angular resolution for each data set. (*fig:continuum-map*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.09402"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\photmoh}{[M/H]_{\rm phot}}$
$\newcommand{\moh}{[M/H]_{\rm phot}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\singlechi}{\chi^2_{\mathrm{single}}}$
$\newcommand{\binarychi}{\chi^2_{\mathrm{binary}}}$
$\newcommand{\chisqimprove}{\log_{10}(\chi^2_{\text{single}} / \chi^2_{\text{binary}})}$
$\newcommand{\Gaia}{\textit{Gaia}}$
$\newcommand{\jd}[1]{\textcolor{blue}{JD:~#1}}$
$\newcommand{\gaia}{\textsl{Gaia}}$</div>



<div id="title">

# Millions of Main-Sequence Binary Stars from Gaia BP/RP Spectra

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.09622-b31b1b.svg)](https://arxiv.org/abs/2507.09622)<mark>Appeared on: 2025-07-15</mark> -  _14 pages, 15 figures, submitted to A&A_

</div>
<div id="authors">

<mark>J. Li</mark>, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>J. Müller-Horn</mark>, <mark>K. El-Badry</mark>, <mark>R. Seeburger</mark>, <mark>X. Zhang</mark>

</div>
<div id="abstract">

**Abstract:** We present an extensive catalog of likely main-sequence binary stars systems (MSMS), derived from Gaia Data Release 3 BP/RP (XP) spectra through the comparison of single- and binary-star model fits. Leveraging the large sample of low-resolution Gaia XP spectra, we use a neural network to build a forward modeling for spectral luminosities of single-stars, as a function of stellar mass and photometric metallicity. Applying this model to XP spctra, we find that this enables the identification of binaries with mass ratios between 0.5 and 1.0 and flux ratios $>0.1$ as "poor" fits to the data, either in spectral shape or flux normalization. From an initial sample of 35 million stars within 1 kpc, we identify 14 million possible binary candidates, and a high-confidence "golden sample" of 1 million binary systems. This large, homogeneous sample of SED-based binaries enables population studies of luminous MSMS binaries, and -- in conjunction with kinematic or astrometric probes -- permits to identify binaries with dark or dim companions, such as white dwarfs, neutron stars and black hole candidates, improving our understanding of compact object populations.

</div>

<div id="div_fig1">

<img src="tmp_2507.09622/./fig_msms/venn.png" alt="Fig6" width="100%"/>

**Figure 6. -** Venn diagram showing the hierarchical relationship between our sample selections.
The parent sample consists of 35 million stars within 1 kpc from Gaia DR3.
From this population, we identify 14 million binary candidates based on spectral model fitting criteria.
The golden sample is identified through selection criteria in Table \ref{tab:golden_sample}.
 (*fig:venn*)

</div>
<div id="div_fig2">

<img src="tmp_2507.09622/./fig_msms/compare_mmsnr50.png" alt="Fig13" width="100%"/>

**Figure 13. -** Validation of parameter recovery for the MSMS mock data. Top row: Comparison between true and inferred values for primary mass ($m_1$, left), $\moh$([M/H], middle), and mass ratio ($q$, right).
    The black dashed lines show the one-to-one relation, while blue points represent individual systems in our validation sample. Inset text shows the median bias and scatter for each parameter. Bottom row: Residuals (inferred minus true values) as a function of the true parameter values, highlighting the consistency of our parameter recovery across the full range of stellar properties.
    Primary masses are recovered with high precision (scatter of 0.049 $M_\odot$), while $\moh$ show broader but still reliable recovery (scatter of 0.192 dex). Mass ratios are determined with good precision (scatter of 0.137) with a small positive bias (0.019) at low $q$ values because of the detection sensitivity limitations.
    The hard limits in the right panel result from enforcing $q < 1$.
     (*fig:compare_qfit*)

</div>
<div id="div_fig3">

<img src="tmp_2507.09622/./fig_msms/gnss_chi2_diff.png" alt="Fig15.1" width="33%"/><img src="tmp_2507.09622/./fig_msms/apogee_chi2_diff.png" alt="Fig15.2" width="33%"/><img src="tmp_2507.09622/./fig_msms/single_chi2_diff.png" alt="Fig15.3" width="33%"/>

**Figure 15. -** Distribution of the logarithmic $\chi^2$ improvement, $\log_{10}(\chi^2_{\text{single}} / \chi^2_{\text{binary}})$, showing the statistical evidence favouring binary over single-star spectral models.
Left panel: Distributions for Gaia DR3 Non-Single Star (NSS) sources: systems with orbital solutions (blue), single-lined spectroscopic binaries (SB1/SB1C; orange), and double-lined spectroscopic binaries (SB2; green).
Right panel: Distributions for specialized populations with likely dark companions: neutron star binaries  ([ and El-Badry 2024]()) (red) and white dwarf-main sequence binaries (WDMS)  ([Li, Ting and Rix 2025]()) (blue). The Gaia DR3 NSS SB2 sample (green dotted line) is shown for reference.
Common elements in each panel: The distribution for all stars within 1 kpc (black dashed line) acts as a control representing the general population. A vertical dashed line marks 0.2 for reference. Higher values of the $\chi^2$ improvement indicate stronger evidence for binarity; known binaries typically show significant positive values. (*fig:chi2_diff_gnss_dark*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.09622"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

141  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

20  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
